# make your session

## to functions use

In [ ]:
access_key_id=''#replace your access key id here
secret_access_key='' #replace your secret access key id here
region = 'eu-west-1' # replace your region
session = boto3.Session(
        aws_access_key_id=access_key_id,
        aws_secret_access_key=secret_access_key,
        region_name=region
    )

# 1 create_db_instance

     """
    Creates a new Amazon RDS database instance using boto3 RDS client.

    Parameters:
    - db_instance_identifier (str): The identifier for the new DB instance.
    - db_name (str): The name of the database to create within the instance.
    - db_user (str): The master username for the DB instance.
    - db_password (str): The master password for the DB instance.
    - db_instance_class (str): The DB instance class/type (default: db.t3.micro).
    - engine (str): The database engine type (default: mysql).
    - allocated_storage (int): The storage capacity allocated for the database in GB (default: 20).
    - publicly_accessible (bool): Whether the DB instance is publicly accessible (default: False).
    - region (str): The AWS region to create the instance in (default: 'us-east-1').

    Returns:
    - response (dict): Response from the create_db_instance API call.
                      Contains information about the created database instance.
    """

In [14]:
import boto3
from botocore.exceptions import ClientError

def create_db_instance(session, db_instance_identifier, db_name, db_user, db_password, db_instance_class='db.t3.micro', engine='mysql', allocated_storage=20, publicly_accessible=False, region='us-east-1'):
    
    # Initialize the RDS client with AWS credentials and region
    rds = session.client('rds')

    # Determine engine type-specific parameters
    if engine == 'mysql':
        db_port = 3306
    elif engine == 'sqlserver-ex':
        db_port = 1433
    else:
        raise ValueError(f"Unsupported engine type: {engine}")

    # Create a new RDS database instance
    try:
        response = rds.create_db_instance(
            DBName=db_name,
            DBInstanceIdentifier=db_instance_identifier,
            AllocatedStorage=allocated_storage,
            DBInstanceClass=db_instance_class,
            Engine=engine,
            MasterUsername=db_user,
            MasterUserPassword=db_password,
            Port=db_port,
            PubliclyAccessible=publicly_accessible,
            MultiAZ=False,  # Change to True for Multi-AZ deployment
            StorageType='gp2',  # Change as per your requirement
            BackupRetentionPeriod=7,  # Change as per your requirement
            LicenseModel='general-public-license',  # Change as per your requirement
            Tags=[
                {'Key': 'Name', 'Value': db_instance_identifier},  # Add tags as needed
            ]
        )
        return response  # Return the response from the create_db_instance API call

    except ClientError as e:
        print(f"Error creating RDS instance: {e}")
        return None


 ### example

In [ ]:
db_instance_identifier = 'irissahani'
db_name = 'irissahani'
db_user = 'admin'
db_password = 'password' 
create_db_instance(session, db_instance_identifier, db_name, db_user, db_password)

# 2 describe_rds_instance 

    """
    Retrieves information about an existing Amazon RDS instance.

    Parameters:
    - instance_id: Identifier for the RDS instance.

    Returns:
    - Dictionary containing detailed information about the RDS instance.
    """

In [94]:
def describe_rds_instance(instance_id, session):
    rds = session.client('rds')
    
    response = rds.describe_db_instances(DBInstanceIdentifier=instance_id)
    return response['DBInstances'][0] if response['DBInstances'] else None


 ### example

In [ ]:
describe_rds_instance("store", session)

# 3 modify_rds_instance

    """
    Modifies an existing Amazon RDS instance.

    Parameters:
    - instance_id: Identifier for the RDS instance to modify.
    - db_instance_class: New RDS instance class (e.g., db.t2.medium).

    Returns:
    - Response object indicating the success of the modification request.
    """

In [96]:
def modify_rds_instance(instance_id, db_instance_class, session):
    rds = session.client('rds')
    
    response = rds.modify_db_instance(
        DBInstanceIdentifier=instance_id,
        DBInstanceClass=db_instance_class,
        ApplyImmediately=True
    )
    return response


 ### example

In [ ]:
instance_id = "shanishani"
new_instance_class = 'db.t3.micro'   # Replace with the new instance class you want to set

    # Call modify_rds_instance function
response = modify_rds_instance(instance_id, new_instance_class, session)

print("Modification response:", response)

# 4 create_read_replica

### Purpose:

##### The create_read_replica function is used to create a read replica of an existing Amazon RDS database instance. A read replica is a copy of the source database instance, and it allows you to offload read-only operations (such as reporting, analytics, or backups) from the primary instance, thereby reducing the load on the primary instance.


    """
    Creates a read replica for an existing Amazon RDS instance.

    Parameters:
    - source_db_instance_id: Identifier of the source RDS instance.
    - replica_db_instance_id: Identifier for the new read replica instance.

    Returns:
    - Response object indicating the success of the read replica creation.
    """

In [ ]:
def create_read_replica(source_db_instance_id, replica_db_instance_id, session):
    rds = session.client('rds')
    
    try:
        response = rds.create_db_instance_read_replica(
            DBInstanceIdentifier=replica_db_instance_id,
            SourceDBInstanceIdentifier=source_db_instance_id,
            # Add more parameters as needed
        )
        return response
    except client.exceptions.ClientError as e:
        print(f"Error creating read replica: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error creating read replica: {e}")
        return None


### Use Cases:
 Scalability: Create read replicas to distribute read traffic 
 and improve overall database performance, especially for read-intensive applications.
 
High Availability: Use read replicas to enhance fault tolerance 
by providing failover options in case the primary instance becomes unavailable.
Reporting and Analytics: Offload reporting queries or analytics workloads to read replicas to avoid impacting the performance of the primary database.

### Considerations:
Replication Lag: Keep in mind that there might be a slight delay (replication lag) 
between changes made to the primary instance and their availability on the read replica.

Cost: While read replicas can improve performance and availability, 
they incur additional costs for storage, data transfer, and maintenance.

This function is essential for managing and optimizing database performance and availability in AWS RDS environments, offering flexibility and scalability as per application requirements.







### Optimization Tips:

1. **Load Balancing Read Traffic**:
   - Utilize read replicas to distribute read queries and balance the load across multiple database instances. By offloading read operations from the primary instance, you can improve overall application performance and responsiveness.

2. **Scaling Read Capabilities**:
   - Scale your application's read capabilities horizontally by creating multiple read replicas. This approach allows you to handle increased read workloads without impacting the performance of the primary database.

3. **Fault Tolerance and High Availability**:
   - Improve fault tolerance and ensure high availability by creating read replicas in different Availability Zones (AZs) or regions. In case of a primary instance failure, read replicas can serve as standby instances, minimizing downtime and ensuring continuity of operations.

4. **Reducing Read Latency**:
   - Place read replicas closer to your application users or clients to reduce read latency. Deploying replicas in different geographical regions or AZs can help optimize response times for distributed applications.

5. **Backup and Recovery**:
   - Use read replicas as backup sources for data recovery and disaster recovery purposes. They provide a point-in-time snapshot of the data from the primary instance, offering an additional layer of data protection.

6. **Scaling for Analytics and Reporting**:
   - Leverage read replicas for running analytics queries, generating reports, or performing data analysis without impacting the performance of the primary database. This separation of workloads ensures that operational tasks and analytical tasks do not compete for resources.

7. **Monitoring and Scaling Automation**:
   - Implement automated monitoring and scaling policies to adjust the number and size of read replicas based on workload demands. AWS provides tools like Amazon CloudWatch and AWS Auto Scaling to automate these processes and optimize resource utilization.

8. **Cost Optimization**:
   - Optimize costs by adjusting the number and size of read replicas based on actual usage patterns. AWS RDS offers flexibility to scale replicas up or down as needed, allowing you to control costs while meeting performance requirements.

9. **Version Upgrades and Testing**:
   - Use read replicas for testing database version upgrades or schema changes before applying them to the primary instance. This approach helps mitigate risks associated with production changes and ensures smooth transitions.

10. **Monitoring Replication Lag**:
    - Monitor replication lag between the primary instance and read replicas to ensure timely data synchronization. Understanding and managing replication lag helps maintain data consistency and integrity across all database instances.

Implementing these optimization tips alongside the `create_read_replica` function can enhance the performance, scalability, and resilience of your Amazon RDS database environment, supporting efficient and cost-effective operations for your applications.

 ### example

In [ ]:
create_read_replica("irisDBShaniS", "replicairisDBShaniS", session)

# 5 enable_automated_backups


## Purpose:

The enable_automated_backups function enables automated backups 
for a specified Amazon RDS instance. Automated backups allow you to recover
your database to any point within your retention period, providing data protection 
against accidental data loss or corruption.

    """
    Enables automated backups for an existing Amazon RDS instance.

    Parameters:
    - instance_id: Identifier of the RDS instance.

    Returns:
    - Response object indicating the success of enabling automated backups.
    """

In [104]:
import boto3

def enable_automated_backups(instance_id, session):
    rds = session.client('rds')
    
    try:
        response = rds.modify_db_instance(
            DBInstanceIdentifier=instance_id,
            BackupRetentionPeriod=7,  # Retain backups for 7 days
            ApplyImmediately=True
        )
        return response
    except rds.exceptions.DBInstanceNotFoundFault as e:
        print(f"Error: RDS instance '{instance_id}' not found.")
        return None
    except rds.exceptions.InvalidDBInstanceStateFault as e:
        print(f"Error: Invalid state for modifying DB instance '{instance_id}': {e}")
        return None
    except client.exceptions.ClientError as e:
        print(f"Error modifying DB instance '{instance_id}': {e}")
        return None
    except Exception as e:
        print(f"Unexpected error enabling automated backups for '{instance_id}': {e}")
        return None




###  Use Cases:

Standardizing backup practices.
Ensuring data durability for critical applications.
Automating backup management to reduce operational overhead.

### Considerations:

Understand cost implications.
Monitor backup performance and compliance.

 ### example

In [ ]:
enable_automated_backups("my-rds-instance-id", session)

Here's how you can structure the information into the requested format:

### Backup and Recovery:
   - Ensure robust backup and recovery capabilities for your RDS instances.
   
### Cost Optimization:
   - Adjust backup retention periods to balance costs with compliance and recovery needs.
   
### Monitoring and Alerting:
   - Implement monitoring to track backup operations and ensure compliance with backup policies.
   
### Disaster Recovery Planning:
   - Use automated backups as part of a comprehensive disaster recovery strategy.
   
### Compliance and Regulatory Requirements:
   - Meet data protection and retention requirements with automated backups.

## 6 create_db_snapshot

    """
    Creates a manual snapshot of an existing Amazon RDS instance.

    Parameters:
    - instance_id: Identifier of the RDS instance to snapshot.
    - snapshot_id: Identifier for the new DB snapshot.

    Returns:
    - Response object indicating the success of the snapshot creation.
    """

In [32]:
def create_db_snapshot(instance_id, snapshot_id, session):
    rds = session.client('rds')
    
    try:
        response = rds.create_db_snapshot(
            DBSnapshotIdentifier=snapshot_id,
            DBInstanceIdentifier=instance_id
        )
        return response
    except client.exceptions.ClientError as e:
        print(f"Error creating DB snapshot: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error creating DB snapshot: {e}")
        return None

 ### example

In [ ]:
create_db_snapshot("store", "storesnapshotstore", session)



## Optimization Tips:

### Backup and Recovery:

1. **Ensure Robust Backup and Recovery Capabilities**:
   - Creating manual snapshots, such as with `create_db_snapshot`, provides a point-in-time backup of your database instance. This backup can be crucial for recovery scenarios, allowing you to restore the database to a specific state in case of data loss or corruption.

2. **Cost Optimization**:
   - Adjust the frequency and retention periods of manual snapshots to balance costs with compliance and recovery needs. AWS charges for snapshot storage based on the volume of data stored, so optimizing snapshot policies can help control expenses.

3. **Monitoring and Alerting**:
   - Implement monitoring to track the creation and status of DB snapshots. Use Amazon CloudWatch alarms or other monitoring tools to ensure that backups are created successfully and meet your backup policy requirements.

4. **Disaster Recovery Planning**:
   - Incorporate manual snapshots into your disaster recovery strategy. Regularly create and validate snapshots to ensure they are available and can be restored quickly in case of a disaster.

5. **Compliance and Regulatory Requirements**:
   - Use manual snapshots to meet data protection and retention requirements mandated by compliance standards or regulatory frameworks. Ensure that snapshots are created and managed in accordance with your organization's data governance policies.




# 7 restore_db_instance_from_snapshot

    """
    Restores an Amazon RDS instance from a specified DB snapshot.

    Parameters:
    - instance_id: Identifier for the new RDS instance.
    - snapshot_id: Identifier of the DB snapshot to restore from.

    Returns:
    - Response object indicating the success of the instance restoration.
    """

In [33]:
def restore_db_instance_from_snapshot(instance_id, snapshot_id, session):
    rds = session.client('rds')

    response = rds.restore_db_instance_from_db_snapshot(
        DBInstanceIdentifier=instance_id,
        DBSnapshotIdentifier=snapshot_id,
        # Add more parameters as needed
    )
    return response


 ### example

In [ ]:
restore_db_instance_from_snapshot("strfrom", "storesnapshotstore", session)

# 8 create_db_parameter_group

    """
    Creates a new DB parameter group for Amazon RDS.

    Parameters:
    - group_name: Name for the new DB parameter group.
    - db_parameter_group_family: Parameter group family name.
    - description: Description of the parameter group.

    Returns:
    - Response object indicating the success of the parameter group creation.
    """

In [39]:
def create_db_parameter_group(group_name, db_parameter_group_family, description, session):
    rds = session.client('rds')
    response = rds.create_db_parameter_group(
        DBParameterGroupName=group_name,
        DBParameterGroupFamily=db_parameter_group_family,
        Description=description
    )
    return response


 ### example

In [ ]:
group_name = 'my-db-parameter-group-shani'
db_parameter_group_family = 'mysql8.0'
description = 'My custom DB parameter group for MySQL 8.0'
create_db_parameter_group(group_name, db_parameter_group_family, description, session)

# 9 modify_db_parameter_group

    """
    Modifies an existing DB parameter group for Amazon RDS.

    Parameters:
    - group_name: Name of the DB parameter group to modify.
    - parameters: List of parameters to modify.

    Returns:
    - Response object indicating the success of the parameter group modification.
    """

In [49]:
def modify_db_parameter_group(group_name, parameters, session):

    rds = session.client('rds')
    
    response = rds.modify_db_parameter_group(
        DBParameterGroupName=group_name,
        Parameters=parameters
    )
    return response


 ### example

In [ ]:
group_name = 'my-db-parameter-group-shani'
parameters = [
        {
            'ParameterName': 'innodb_buffer_pool_size',
            'ParameterValue': '134217728',
            'ApplyMethod': 'pending-reboot'
        },
        {
            'ParameterName': 'max_connections',
            'ParameterValue': '1000',
            'ApplyMethod': 'immediate'
        }
        # Add more parameters as needed
    ]
modify_db_parameter_group(group_name, parameters, session)

# 10 delete_rds_instance

    """
    Deletes an existing Amazon RDS instance.

    Parameters:
    - instance_id: Identifier of the RDS instance to delete.
    - skip_final_snapshot: Whether to skip creating a final snapshot before deletion.

    Returns:
    - Response object indicating the success of the instance deletion.
    """

In [51]:
def delete_rds_instance(instance_id, skip_final_snapshot=True, session):
    
    rds = session.client('rds')
    

    response = rds.delete_db_instance(
        DBInstanceIdentifier=instance_id,
        SkipFinalSnapshot=skip_final_snapshot
    )
    return response


 ### example

In [ ]:
delete_rds_instance("store", session)

# 11 create_db_cluster_parameter_group

    """
    Creates a new DB cluster parameter group in Amazon RDS.

    Parameters:
    - db_cluster_parameter_group_name (str): The name for the new DB cluster parameter group.
    - db_parameter_group_family (str): The family of the DB cluster parameter group.
    - description (str): The description for the new DB cluster parameter group.
    - tags (list of dict): Tags to associate with the DB cluster parameter group.
                           Each tag is a dictionary with 'Key' and 'Value'.
    - session (boto3.Session): The boto3 Session object with AWS credentials and region.

    Returns:
    - response (dict): Response from the create_db_cluster_parameter_group API call.
                      Contains information about the created DB cluster parameter group.
    """

In [17]:
import boto3

def create_db_cluster_parameter_group(db_cluster_parameter_group_name, db_parameter_group_family, description, tags, session):

    rds = session.client('rds', region_name=session.region_name)
    try:
        response = rds.create_db_cluster_parameter_group(
            DBClusterParameterGroupName=db_cluster_parameter_group_name,
            DBParameterGroupFamily=db_parameter_group_family,
            Description=description,
            Tags=tags
        )
        return response
    except rds.exceptions.DBParameterGroupAlreadyExistsFault as e:
        print(f"Error: DB cluster parameter group '{db_cluster_parameter_group_name}' already exists.")
        return None
    except Exception as e:
        print(f"Error creating DB cluster parameter group '{db_cluster_parameter_group_name}': {e}")
        return None



 ### example

In [19]:
db_cluster_param_group_name = 'my-cluster-param-group1'
db_param_group_family = 'aurora-mysql5.7'
description = 'Parameter group for Aurora MySQL 5.7'
tags = [{'Key': 'Environment', 'Value': 'Production'}]
response = create_db_cluster_parameter_group(db_cluster_param_group_name, db_param_group_family, description, tags, session)
if response:
    print("DB cluster parameter group created successfully.")
    # Optionally, handle response details as needed
else:
    print("Failed to create DB cluster parameter group.")


DB cluster parameter group created successfully.


# 12 describe_db_replicas

    """
    Retrieves information about DB replicas (read replicas) using boto3.

    Parameters:
    - session (boto3.Session): Boto3 session object with AWS credentials and region.

    Returns:
    - replicas (list of dict): List of DB replica instances with their details.
                              Each replica is represented as a dictionary.
                              Returns None if no replicas are found or if an error occurs.
    """

In [30]:
import boto3

def describe_db_replicas(session):

    try:
        # Initialize the RDS client with the provided session
        rds = session.client('rds')

        # Describe DB instances to retrieve all instances including replicas
        response = rds.describe_db_instances()

        # Check if 'DBInstances' key exists in response
        if 'DBInstances' in response:
            # Filter out replicas by checking if 'ReadReplicaSourceDBInstanceIdentifier' exists
            replicas = [db_instance for db_instance in response['DBInstances'] if db_instance.get('ReadReplicaSourceDBInstanceIdentifier')]

            # Print details of each replica found
            for replica in replicas:
                print(f"Replica ID: {replica['DBInstanceIdentifier']}, Source: {replica['ReadReplicaSourceDBInstanceIdentifier']}")

            return replicas
        else:
            print("No 'DBInstances' key found in response.")
            return None

    except Exception as e:
        print(f"Error describing DB replicas: {e}")
        return None


 ### example

In [ ]:
describe_db_replicas(session)